# Statystyczne Reguły Decyzyjne
## Projekt: analiza zbioru Adult
Laura Hoang 140381

Jan Wojnowski 140701

Palina Ionina 140200

### 1. Wprowadzanie i opis problemu

W ramach niniejszego projektu rozważany jest problem klasyfikacji binarnej, polegający na przewidywaniu poziomu dochodu osoby fizycznej na podstawie danych pochodzących ze spisu ludności. Do realizacji zadania wykorzystano klasyczny zbiór danych **Adult (Census Income)**, który stanowi standardowy punkt odniesienia w badaniach nad metodami klasyfikacyjnymi oraz regułami decyzyjnymi.

Rozpatrywany problem decyzyjny polega na **przewidywaniu, czy roczny dochód danej osoby przekracza 50 000 dolarów amerykańskich**, na podstawie zestawu jej cech demograficznych i ekonomicznych, pochodzących z danych spisowych. Formalnie jest to zadanie **klasyfikacji binarnej**, w którym każdej obserwacji przypisywana jest jedna z dwóch możliwych klas decyzyjnych:

* dochód **nieprzekraczający 50 000 USD rocznie** (`<=50K`),
* dochód **przekraczający 50 000 USD rocznie** (`>50K`).

Zmienna decyzyjna `income` stanowi etykietę klasy, natomiast pozostałe zmienne opisowe tworzą przestrzeń cech, na podstawie której konstruowany jest model klasyfikacyjny. Obejmują one m.in. wiek, poziom wykształcenia, formę zatrudnienia, wykonywany zawód, stan cywilny, liczbę przepracowanych godzin tygodniowo, a także cechy demograficzne takie jak płeć, rasa czy kraj pochodzenia.

Celem analizy jest wyznaczenie takiej reguły decyzyjnej, która na podstawie obserwowanego wektora cech maksymalizuje prawdopodobieństwo poprawnej klasyfikacji dochodu danej osoby. Problem ten ma istotne znaczenie praktyczne, ponieważ pozwala analizować zależności pomiędzy cechami demograficznymi a poziomem dochodów, a jednocześnie stanowi dobre studium przypadku do porównywania różnych metod klasyfikacji statystycznej.

Z punktu widzenia teorii statystycznych reguł decyzyjnych, zadanie to może być rozpatrywane jako problem minimalizacji ryzyka klasyfikacyjnego przy zadanej funkcji straty, co umożliwia formalną ocenę jakości zastosowanych modeli oraz porównanie ich skuteczności.

In [1]:
# pobranie potrzebnych bibliotek
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ucimlrepo import fetch_ucirepo  # biblioteka z danymi

import warnings
warnings.filterwarnings("ignore")

In [ ]:
adult = fetch_ucirepo(id=2)  # zbiór danych

### 2. Opis zbioru danych

W projekcie wykorzystano zbiór danych **Adult** (znany również jako **Census Income Dataset**), pochodzący z repozytorium **UCI Machine Learning Repository**. Zbiór ten został opracowany na podstawie danych spisowych z **1994 roku**, a jego ekstrakcję przeprowadzili **Barry Becker** oraz **Ronny Kohavi**. Dane zostały opublikowane w **1996 roku**, a ich najnowsza aktualizacja miała miejsce **24 września 2024 roku**. Zbiór posiada identyfikator DOI: *10.24432/C5XW20*.

Celem zbioru danych jest rozwiązanie zadania **klasyfikacji binarnej**, polegającego na przewidywaniu, czy **roczny dochód danej osoby przekracza 50 000 USD**. Zmienna decyzyjna (`income`) przyjmuje dwie wartości: `>50K` oraz `<=50K`.

Zbiór danych należy do obszaru **nauk społecznych** i ma charakter **wielowymiarowy (multivariate)**. Składa się z **48 842 obserwacji** oraz **14 zmiennych objaśniających**, opisujących cechy demograficzne i ekonomiczne badanych osób. Zmienne mają charakter **liczbowy (integer)** oraz **kategoryczny** i obejmują informacje dotyczące m.in. wieku, poziomu wykształcenia, sytuacji zawodowej, stanu cywilnego, rasy, płci oraz liczby przepracowanych godzin w tygodniu.

Podczas ekstrakcji danych zastosowano zestaw warunków filtrujących w celu uzyskania względnie jednorodnego i „czystego” zbioru obserwacji. Do zbioru włączono jedynie rekordy spełniające następujące kryteria:

* wiek powyżej 16 lat,
* dochód brutto powyżej 100 USD,
* waga populacyjna (`fnlwgt`) większa od 1,
* liczba godzin pracy w tygodniu większa od 0.

W zbiorze danych występują **braki danych**, oznaczone symbolem `NaN`, głównie w zmiennych kategorycznych takich jak `workclass`, `occupation` oraz `native-country`.

**Rozkład zmiennej celu:**
Zbiór charakteryzuje się umiarkowanym niezbalansowaniem klas. Klasa mniejszościowa (`>50K`) stanowi około **24%** obserwacji, podczas gdy klasa większościowa (`<=50K`) obejmuje około **76%** rekordów. Ta asymetria zostanie uwzględniona przy wyborze metryk oceny modeli.

| Zmienna            | Rola         | Typ danych   | Demografia           | Opis wartości                                                    | Braki danych |
| ------------------ | ------------ | ------------ | -------------------- | ---------------------------------------------------------------- | ------------ |
| **age**            | Cecha        | Liczbowa     | Wiek                 | *(brak)*                                                         | nie          |
| **workclass**      | Cecha        | Kategoryczna | Zarobki              | Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov… | tak          |
| **fnlwgt**         | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **education**      | Cecha        | Kategoryczna | Poziom Wykształcenia | Bachelors, Some-college, 11th, HS-grad, Prof-school, …           | nie          |
| **education-num**  | Cecha        | Liczbowa     | Poziom Wykształcenia | *(brak)*                                                         | nie          |
| **marital-status** | Cecha        | Kategoryczna | Inne                 | Married-civ-spouse, Divorced, Never-married, Separated, …        | nie          |
| **occupation**     | Cecha        | Kategoryczna | Inne                 | Tech-support, Craft-repair, Other-service, Sales, …              | tak          |
| **relationship**   | Cecha        | Kategoryczna | Inne                 | Wife, Own-child, Husband, Not-in-family, Other-relative, …       | nie          |
| **race**           | Cecha        | Kategoryczna | Rasa                 | White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black      | nie          |
| **sex**            | Cecha        | Binarna      | Płeć                 | Female, Male                                                     | nie          |
| **capital-gain**   | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **capital-loss**   | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **hours-per-week** | Cecha        | Liczbowa     | *(brak)*             | *(brak)*                                                         | nie          |
| **native-country** | Cecha        | Kategoryczna | Inne                 | United-States, Cambodia, England, Puerto-Rico, Canada, …         | tak          |
| **income**         | Zmienna celu | Binarna      | Zarobki              | >50K, <=50K                                                      | nie          |


In [ ]:
X = adult.data.features
y = adult.data.targets

df_adult = X.copy()
df_adult["income"] = y
df_adult.head()

### 3. Czyszczenie i wstępne przetwarzanie danych
Czyszczenie i wstępne przetwarzanie danych - imputacja braków danych, standaryzacja, kodowanie typu one-hot, transformacja wartości odstających, itp.

In [ ]:
df_adult.info()

#### 3.1. Identyfikacja i analiza braków danych

Wstępna weryfikacja zbioru danych wykazała obecność niestandardowych oznaczeń braków danych (znak `?`) w zmiennych kategorycznych: `workclass`, `occupation` oraz `native-country`. Przed przystąpieniem do analizy dokonano unifikacji oznaczeń do formatu `NaN`. Poniżej przedstawiono statystyki braków oraz zbadano ich wpływ na zmienną celu, aby dobrać odpowiednią metodę imputacji.


In [ ]:
cols_wmiss = ['workclass', 'occupation', 'native-country']

# Standaryzacja braków danych (zamiana '?' na np.bnan)
df_adult = df_adult.replace('?', np.nan)

missing_stats = df_adult[cols_wmiss].isnull().sum().to_frame(name='Liczba braków')
missing_stats['Procent braków'] = (missing_stats['Liczba braków'] / len(df_adult)) * 100
print("Statystyki braków danych przed imputacją:")
print(missing_stats)

overlap = (df_adult['workclass'].isna() & df_adult['occupation'].isna()).sum()
print(f"\nLiczba rekordów z brakiem jednocześnie w 'workclass' i 'occupation': {overlap}")

# Weryfikacja nielosowości braków
mask_missing_workclass = df_adult['workclass'].isna()

pct_miss = (df_adult[mask_missing_workclass]['income'] == '>50K').mean() * 100
pct_nomiss = (df_adult[~mask_missing_workclass]['income'] == '>50K').mean() * 100

print(f"\nAnaliza wpływu braków na zmienną celu (income >50K):")
print(f" - Grupa z brakami w workclass: {pct_miss:.2f}%")
print(f" - Grupa bez braków:            {pct_nomiss:.2f}%")
print(" Wniosek: Braki są skorelowane z niższym dochodem (MNAR).")


**Wnioski i decyzja o imputacji**

Przeprowadzona analiza wykazała silną współzależność między brakami w zmiennych `workclass` i `occupation`. Co ważniejsze, analiza rozkładu zmiennej decyzyjnej ujawniła, że grupa osób z brakującymi danymi charakteryzuje się znacznie niższym odsetkiem osób zamożnych (>50K) w porównaniu do reszty populacji.

Sugeruje to, że braki te nie są błędem losowym, lecz niosą istotną informację semantyczną – najprawdopodobniej dotyczą osób bezrobotnych lub nigdy niepracujących. Z tego względu odrzucono metody usuwania obserwacji oraz imputacji najczęstszą wartością. Zdecydowano się na **utworzenie nowej kategorii 'Unknown'**, co pozwoli modelowi wykorzystać informację o braku danych jako cechę predykcyjną.


In [ ]:
# Implementacja decyzji: Imputacja stałą wartością 'Unknown'
for col in cols_wmiss:
    df_adult[col] = df_adult[col].fillna('Unknown')

print("Imputacja zakończona pomyślnie. Nowa liczba braków:")
print(df_adult[cols_wmiss].isnull().sum())

#### 3.2. Kodowanie zmiennych kategorycznych
W zbiorze zawartych jest 7 zmiennych kategorycznych: `workclass`, `education`, `marital-status`, `occupation`, `relationship`, `race`, `sex`, `native-country`.

##### 3.2.1. Zmienna workclass

In [ ]:
# Sprawdzenie liczności kategorii w workclass
wc_counts = df_adult['workclass'].value_counts()
print(wc_counts)

# Sprawdzenie udziału procentowego
print("\nUdział procentowy:")
print(df_adult['workclass'].value_counts(normalize=True) * 100)

Zmienna `workclass` posiada 9 unikalnych kategorii (wliczając `Unknown`). Analiza rozkładu wskazuje na silną dominację sektora prywatnego oraz występowanie klas o marginalnej liczności takich jak `Without-pay` czy `Never-worked`. Mimo dużego niezbalansowania, zdecydowano się zachować pełną szczegółowość danych i przekształcić zmienną metodą One-Hot Encoding.

In [ ]:
# Kodowanie zmiennej workclass
df_adult = pd.get_dummies(df_adult, columns=['workclass'], drop_first=True, dtype=int)
df_adult.head()

##### 3.2.2. Zmienna education
Zbiór zawiera dwie zmienne o tym samym znaczeniu: `education` oraz `education-num`.

In [ ]:
df_adult.loc[:, ['education', 'education-num']].drop_duplicates().sort_values(by='education-num')

Zmienna `education-num` zachowuje porządkowość zmiennej `education`, zatem usunięta zostanie ta druga.

In [ ]:
df_adult = df_adult.drop(columns=['education'])

##### 3.2.3. Zmienna marital-status

In [ ]:
# Sprawdzenie liczności kategorii w marital-status
ms_counts = df_adult['marital-status'].value_counts()
print(ms_counts)

# Sprawdzenie udziału procentowego
print("\nUdział procentowy:")
print(df_adult['marital-status'].value_counts(normalize=True) * 100)

In [ ]:
df_adult = pd.get_dummies(df_adult, columns=['marital-status'], drop_first=True, dtype=int)
df_adult.head()

##### 3.2.4. Zmienna relationship

In [ ]:
# Sprawdzenie liczności kategorii w relationship
ms_counts = df_adult['relationship'].value_counts()
print(ms_counts)

# Sprawdzenie udziału procentowego
print("\nUdział procentowy:")
print(df_adult['relationship'].value_counts(normalize=True) * 100)

In [ ]:
df_adult = pd.get_dummies(df_adult, columns=['relationship'], drop_first=True, dtype=int)
df_adult.head()

##### 3.2.5. Zmienna occupation

In [ ]:
# Sprawdzenie liczności kategorii w occupation
oc_counts = df_adult['occupation'].value_counts()
print(oc_counts)

# Sprawdzenie udziału procentowego
print("\nUdział procentowy:")
print(df_adult['occupation'].value_counts(normalize=True) * 100)

In [ ]:
df_adult = pd.get_dummies(df_adult, columns=['occupation'], drop_first=True, dtype=int)
df_adult.head()

##### 3.2.6. Zmienna race

In [ ]:
# Sprawdzenie liczności kategorii w race
ra_counts = df_adult['race'].value_counts()
print(ra_counts)

# Sprawdzenie udziału procentowego
print("\nUdział procentowy:")
print(df_adult['race'].value_counts(normalize=True) * 100)

In [ ]:
df_adult = pd.get_dummies(df_adult, columns=['race'], drop_first=True, dtype=int)
df_adult.head()

##### 3.2.7. Zmienna sex

In [ ]:
# Sprawdzenie liczności kategorii w sex
sex_counts = df_adult['sex'].value_counts()
print(sex_counts)

# Sprawdzenie udziału procentowego
print("\nUdział procentowy:")
print(df_adult['sex'].value_counts(normalize=True) * 100)

In [ ]:
df_adult['is_male'] = df_adult['sex'].apply(lambda x: int(x=='Male'))
df_adult = df_adult.drop(columns=['sex'])
df_adult

##### 3.2.8. Zmienna native-country

In [ ]:
# Sprawdzenie liczności kategorii w native-country
nc_counts = df_adult['native-country'].value_counts()
print(nc_counts)

# Sprawdzenie udziału procentowego
print("\nUdział procentowy:")
print(df_adult['native-country'].value_counts(normalize=True) * 100)

Zmienna `native-country` posiada 42 unikalne kategorie, z czego **United-States** dominuje z udziałem niemal 90%. Tak duża dysproporcja sprawia, że kodowanie One-Hot wygenerowałoby wiele rzadkich (sparse) kolumn o marginalnej wartości predykcyjnej. Z tego względu zdecydowano się na **uproszczenie zmiennej do postaci binarnej**: USA vs pozostałe kraje.

In [ ]:
# Kodowanie binarne: 1 dla USA, 0 dla pozostałych
df_adult['native-country_US'] = df_adult['native-country'].apply(lambda x: 1 if x == 'United-States' else 0)

# Usunięcie oryginalnej kolumny tekstowej
df_adult = df_adult.drop(columns=['native-country'])

# Weryfikacja
print("Rozkład po kodowaniu (1=USA, 0=Other):")
print(df_adult['native-country_US'].value_counts(normalize=True) * 100)

##### 3.2.9. Zmienna celu income

In [ ]:
df_adult['income'].value_counts()

In [ ]:
df_adult['income_binary'] = df_adult['income'].apply(lambda x: 1 if x in ('>50K', '>50K.') else 0)
df_adult = df_adult.drop(columns=['income'])
df_adult.head()

In [ ]:
df_adult.info()

#### 3.3. Usunięcie zmiennej `fnlwgt`

Zmienna `fnlwgt` (final weight) jest **wagą demograficzną** przypisaną przez Census Bureau. Określa ona, ile osób w całej populacji USA reprezentuje dany rekord w próbie. Jest to zmienna techniczna służąca do ważenia wyników statystycznych na poziomie populacji, a **nie cecha opisująca indywidualną osobę**. W związku z tym zdecydowano się ją usunąć ze zbioru cech.

In [ ]:
df_adult = df_adult.drop(columns=['fnlwgt'])

#### 3.4. Standaryzacja zmiennych numerycznych

Zmienne numeryczne w zbiorze (`age`, `education-num`, `capital-gain`, `capital-loss`, `hours-per-week`) posiadają różne skale i rozkłady. Dla algorytmów opartych na gradientach (w tym regresji logistycznej) zalecana jest standaryzacja cech do porównywalnej skali. Zastosowano standaryzację z-score (StandardScaler), która przekształca każdą zmienną tak, aby miała średnią równą 0 i odchylenie standardowe równe 1. Standaryzacja zostanie zastosowana tylko na zbiorze treningowym, a następnie te same parametry zostaną użyte do transformacji zbioru testowego, aby uniknąć wycieku danych (data leakage).

In [ ]:
from sklearn.preprocessing import StandardScaler

num_cols = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = StandardScaler()

### 4. Eksploracyjna analiza danych
W tej sekcji przeprowadzono analizę statystyczną i wizualną przetworzonych danych. Celem jest zrozumienie rozkładów zmiennych, zidentyfikowanie cech różnicujących grupy dochodowe oraz zbadanie siły korelacji między zmiennymi a zmienną celową.

#### 4.1. Rozkłady zmiennych numerycznych
W pierwszej kolejności przeanalizowano rozkłady zmiennych ciągłych i porządkowych. Z analizy wyłączono zmienne binarne (posiadające tylko 2 wartości), skupiając się na cechach o większej różnorodności wartości.

In [ ]:
for col in ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']:
    plt.figure(figsize=(6, 4))
    plt.hist(df_adult[col], bins=30, edgecolor='black', alpha=0.7)
    plt.title(f'Rozkład zmiennej: {col}')
    plt.xlabel(col)
    plt.ylabel('Liczebność')
    plt.grid(axis='y', alpha=0.5)
    plt.show()

**Wnioski z analizy rozkładów:**

Wygenerowane histogramy ujawniają kluczowe charakterystyki populacji:
1.  **age:** Rozkład jest prawoskośny (przesunięty w stronę młodszych uczestników), z wyraźną dominacją osób w wieku produkcyjnym (20-50 lat).
2.  **education-num:** Rozkład wielomodalny. Wyraźne "piki" (słupki) odpowiadają momentom kończenia poszczególnych etapów edukacji (np. liceum - 9, licencjat - 13).
3.  **capital-gain / capital-loss:** Zmienne ekstremalnie skośne. Zdecydowana większość słupka znajduje się przy wartości 0, co oznacza, że większość ludzi nie ma zysków/strat kapitałowych.
4.  **hours-per-week:** Dominująca wartość (moda) to 40 godzin, co tworzy wielki słupek pośrodku wykresu, odzwierciedlając standardowy etat pracy.

#### 4.2. Rozkłady zmiennych kategorycznych

Poniżej przedstawiono rozkłady zmiennych kategorycznych w oryginalnym zbiorze danych (przed kodowaniem one-hot). Analiza ta pozwala zrozumieć strukturę populacji oraz zidentyfikować kategorie dominujące i rzadkie.

In [ ]:
# Używamy oryginalnych danych z ucimlrepo (przed przetworzeniem)
X_original = adult.data.features

for col in ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']:
    num_vals = len(X_original[col].dropna().unique())
    plt.figure(figsize=(max(num_vals * 1.2, 8), 5))
    counts = X_original[col].value_counts()
    plt.bar(x=range(len(counts)), height=counts.values, edgecolor='black', alpha=0.7)
    plt.title(f'Rozkład zmiennej: {col}')
    plt.xticks(range(len(counts)), counts.index, rotation=45, ha='right')
    plt.ylabel('Liczebność')
    plt.grid(axis='y', alpha=0.5)
    plt.tight_layout()
    plt.show()

#### 4.3. Rozkład zmiennej celu: income
Zmienna celu 'income' jest niezbalansowana. 

In [ ]:
target_names = ['<=50K', '>50K']

In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(x=df_adult['income_binary'].value_counts().index, height=df_adult['income_binary'].value_counts().values, edgecolor='black', alpha=0.7)
plt.xticks(df_adult['income_binary'].value_counts().index, labels=target_names)
plt.title(f'Rozkład zmiennej celu: income')
plt.grid(axis='y', alpha=0.5)
plt.show()

In [ ]:
print(f"Stosunek klasy '<=50K' do '>50K': {df_adult['income_binary'].value_counts()[0] / df_adult['income_binary'].value_counts()[1]}")

#### 4.4. Analiza korelacji
W celu znalezienia zależności między zmiennymi, wygenerowana została poniżej macierz korelacji. Ze względu na występowanie zmiennyh binarnych i kategorycznych, wykorzystana została korelacja rank Spearmana.

In [ ]:
matrix = df_adult.corr(method='spearman')

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(matrix, xticklabels=matrix.columns.values, yticklabels=matrix.columns.values, annot=True, fmt='.1f', annot_kws={"size":6});
plt.show()

**Wnioski z analizy korelacji:**

**Korelacje między zmiennymi niezależnymi:**
- `workclass_Unknown` i `occupation_Unknown` = 1.0 (te same osoby mają braki w obu zmiennych)
- `marital-status_Married-civ-spouse` i `relationship_Husband` = 0.9 (naturalna zależność semantyczna)
- `race_Black` i `race_White` = -0.8 (wzajemne wykluczanie kategorii)

**Korelacje z zmienną celu (income_binary):**
- `marital-status_Married-civ-spouse` ~ 0.4 (osoby zamężne/żonate zarabiają więcej)
- `education-num` ~ 0.3 (wyższe wykształcenie = wyższy dochód)
- `age` ~ 0.2 (starsi zarabiają więcej)
- `hours-per-week` ~ 0.2 (więcej godzin pracy = wyższy dochód)
- `relationship_Own-child` ~ -0.2 (dzieci mieszkające z rodzicami zarabiają mniej)
- `marital-status_Never-married` ~ -0.3 (osoby niezamężne zarabiają mniej)


#### 4.5. Analiza zależności zmiennych od zmiennej celu

W tej sekcji zbadano, jak poszczególne zmienne różnicują grupy dochodowe. Celem jest identyfikacja cech o największej mocy dyskryminacyjnej, które będą kluczowe dla budowanych modeli klasyfikacyjnych.

##### 4.5.1. Zmienne numeryczne vs dochód

Wykresy pudełkowe (boxploty) pozwalają porównać rozkłady zmiennych numerycznych między grupami dochodowymi. Różnice w medianach i rozstępach międzykwartylowych wskazują na potencjalną moc predykcyjną zmiennej.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, col in enumerate(['age', 'education-num', 'hours-per-week']):
    sns.boxplot(x='income_binary', y=col, data=df_adult, ax=axes[i], hue='income_binary', palette='Set2', legend=False)
    axes[i].set_title(f'{col} vs dochód')
    axes[i].set_xticklabels(['<=50K', '>50K'])
    axes[i].set_xlabel('Dochód')

plt.tight_layout()
plt.show()

**Wnioski z analizy boxplotów:**

1. **age:** Mediana wieku dla grupy >50K (~44 lata) jest wyraźnie wyższa niż dla <=50K (~34 lata). Wyższy dochód wiąże się z większym doświadczeniem zawodowym.

2. **education-num:** Najsilniejsza separacja klas. Mediana dla >50K to ~13 (licencjat), dla <=50K to ~9 (szkoła średnia). Wykształcenie jest kluczowym predyktorem dochodu.

3. **hours-per-week:** Osoby zarabiające >50K pracują więcej (mediana ~45h) niż grupa <=50K (mediana ~40h). Wyższy dochód koreluje z większym zaangażowaniem czasowym.

##### 4.5.2. Zmienne kategoryczne vs dochód

Wykresy słupkowe przedstawiają odsetek osób zarabiających >50K w poszczególnych kategoriach. Pozwala to zidentyfikować grupy o najwyższym i najniższym prawdopodobieństwie wysokiego dochodu.

In [ ]:
# Przygotowanie danych do analizy (oryginalne dane + zmienna celu)
df_analysis = adult.data.features.copy()
df_analysis['income'] = adult.data.targets
df_analysis['income_binary'] = df_analysis['income'].apply(lambda x: 1 if x in ('>50K', '>50K.') else 0)

cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'sex']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(cat_cols):
    # Oblicz % osób z >50K dla każdej kategorii
    pct_high_income = df_analysis.groupby(col)['income_binary'].mean().sort_values(ascending=False)
    
    axes[i].barh(range(len(pct_high_income)), pct_high_income.values, color='steelblue', edgecolor='black')
    axes[i].set_yticks(range(len(pct_high_income)))
    axes[i].set_yticklabels(pct_high_income.index)
    axes[i].set_xlabel('Odsetek >50K')
    axes[i].set_title(f'Odsetek >50K wg {col}')
    axes[i].axvline(x=0.24, color='red', linestyle='--', label='Średnia (24%)')
    axes[i].set_xlim(0, 0.8)

# Ukryj ostatni pusty subplot
axes[5].axis('off')

plt.tight_layout()
plt.show()

**Wnioski z analizy zmiennych kategorycznych:**

1. **education:** Najsilniejszy predyktor. Osoby z wykształceniem Prof-school (~75%) i Doctorate (~70%) mają najwyższe szanse na dochód >50K. Osoby bez matury (Preschool, 1st-4th) praktycznie nie osiągają wysokich dochodów.

2. **marital-status:** Osoby w związku małżeńskim (Married-civ-spouse) mają ~45% szans na >50K, podczas gdy osoby nigdy niezamężne tylko ~5%.

3. **occupation:** Exec-managerial i Prof-specialty mają najwyższy odsetek (~50%), podczas gdy prywatna służba domowa (Priv-house-serv) i rolnictwo mają najniższy.

4. **sex:** Mężczyźni mają znacznie wyższy odsetek >50K (~30%) niż kobiety (~11%), co odzwierciedla historyczne nierówności płacowe.

5. **workclass:** Self-emp-inc (samozatrudnieni z inkorporacją) mają najwyższy odsetek (~55%), co sugeruje, że przedsiębiorcy osiągają wyższe dochody.

### 5. Budowa modeli
W niniejszej sekcji przedstawiono proces budowy oraz strojenia modeli statystycznych zastosowanych do rozwiązania problemu klasyfikacji, zgodnie z założeniami projektu obejmującymi konstrukcję co najmniej trzech modeli oraz dobór ich hiperparametrów w celu uzyskania możliwie najlepszej jakości predykcji.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

from imblearn.over_sampling import SMOTE

from sklearn.metrics import confusion_matrix, auc, roc_curve, classification_report, accuracy_score, precision_score, recall_score, f1_score

# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import l1_min_c

import random
random.seed(42)

In [ ]:
# podział zbioru na treningowy i testowy 
X = df_adult.drop(columns=['income_binary'])
y = df_adult['income_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standaryzacja zmiennych numerycznych (fit na train, transform na train i test)
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

Ze względu na nierównowagę w liczebości klas zmiennej celu, można przetestować SMOTE

In [ ]:
smote = SMOTE(random_state=42);
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train);

Funkcje pomocnicze

In [ ]:
def plot_confusion_matrices(y_train, y_train_hat, y_test, y_test_hat):
    cm_train, cm_test = confusion_matrix(y_train, y_train_hat), confusion_matrix(y_test, y_test_hat)

    plt.figure(figsize=(10, 4))

    plt.subplot(1,2,1)
    sns.heatmap(cm_train, annot=True, fmt="d", cmap="Blues", 
                xticklabels=["Predicted 0", "Predicted 1"], 
                yticklabels=["Actual 0", "Actual 1"])
    plt.title("Train Confusion Matrix")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

    plt.subplot(1,2,2)
    sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues", 
                xticklabels=["Predicted 0", "Predicted 1"], 
                yticklabels=["Actual 0", "Actual 1"])
    plt.title("Test Confusion Matrix")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

    plt.show()


def plot_roc_auc_curve(X_train, y_train, X_test, y_test, model):
    y_train_lr_proba = model.predict_proba(X_train)[:,1]
    fprt, tprt, _ = roc_curve(y_train, y_train_lr_proba)
    auc_roct = auc(fprt, tprt)

    y_test_lr_proba = model.predict_proba(X_test)[:,1]
    fprv, tprv, _ = roc_curve(y_test, y_test_lr_proba)
    auc_rocv = auc(fprv, tprv)

    plt.figure(figsize=(5, 4))

    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC+AUC")

    plt.plot([0, 1], [0, 1], color="grey", linestyle="--", label="Random, AUC = 0.5")
    plt.plot([0, 0], [0, 1], color="navy", linestyle=":", label="Wizard, AUC = 1.0")
    plt.plot([0, 1], [1, 1], color="navy", linestyle=":")

    plt.plot(fprt, tprt, color="orange", label="Model - train, AUC = %0.2f" % auc_roct)
    plt.plot(fprv, tprv, color="red", label="Model - val, AUC = %0.2f" % auc_rocv)
    plt.legend(loc="lower right");

    plt.show()


def print_report(y_train, y_train_hat, y_test, y_test_hat, target_names):
    print("Training data:\n", classification_report(y_train, y_train_hat, target_names=target_names), "\n\nTest data:\n", classification_report(y_test, y_test_hat, target_names=target_names))

#### 5.1. Regresja logistyczna

In [ ]:
from sklearn.linear_model import LogisticRegression

##### 5.1.1. Podstawowy model - baseline
Ze względu na niezbalnsowanie zmiennej celu, zoastosowane zostaną 2 podejścia: SMOTE oraz class weights.

**Regresja logistyczna + SMOTE**

In [ ]:
lr_smote = LogisticRegression(random_state=42)
lr_smote.fit(X_train_smote, y_train_smote)

In [ ]:
y_train_lr_smote = lr_smote.predict(X_train)
y_test_lr_smote = lr_smote.predict(X_test)

In [ ]:
plot_confusion_matrices(y_train, y_train_lr_smote, y_test, y_test_lr_smote)
plot_roc_auc_curve(X_train, y_train, X_test, y_test, lr_smote)
print_report(y_train, y_train_lr_smote, y_test, y_test_lr_smote, target_names)

**Regresja logistyczna + class weight**

In [ ]:
# model ze zastosowaniem wag klas
lr_clas_weights = LogisticRegression(random_state=42, class_weight='balanced')
lr_clas_weights.fit(X_train, y_train)

In [ ]:
y_train_lr_cw = lr_clas_weights.predict(X_train)
y_test_lrcw = lr_clas_weights.predict(X_test)

In [ ]:
plot_confusion_matrices(y_train, y_train_lr_cw, y_test, y_test_lrcw)
plot_roc_auc_curve(X_train, y_train, X_test, y_test, lr_clas_weights)
print_report(y_train, y_train_lr_cw, y_test, y_test_lrcw, target_names)

##### 5.1.2. Dobór parametrów do modelu
Przy pomocy Grid Search dobrane zostaną wartości:
- C: od 0 do 10
- penalty: brak, L1, L2, L1+L2 (elastic net)

Ze względu na występowanie zmiennych binarnych (na wskutek one-hot encoding), najlepszym solverem będzie 'saga'.

In [ ]:
lr = LogisticRegression(solver="saga", class_weight='balanced', max_iter=1000, random_state=42)

C_grid = np.logspace(-3, 1, 7) # np.linspace(0.01, 10, 100)
penalty = [None] # [None, "l1", "l2", "elasticnet"]
param_grid = {"penalty": penalty, "C": C_grid}

In [ ]:
grid_search = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring="accuracy",
    cv=8,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

In [ ]:
print("GridSearchCV best parameters")
print(f"C:\t\t{grid_search.best_params_['C']}")
print(f"penalty:\t{grid_search.best_params_['penalty']}")

print("\nBest scores")
print(f"GridSearch:\t{grid_search.best_score_:.10f}")

#### 5.2. Las losowy

#### 5.3. XGBoost

### 6. Ocena i wybór najlepszego modelu
Graficzna i opisowa ocena oraz wybór modelu

### 7. Podsumowanie
Podsumowanie wyników, dyskusja na temat napotkanych problemów, wyzwań i zastosowanych rozwiązań